<div class="alert alert-warning">
    BETA RELEASE
    
    This project remains in the experimental stage and there are likely many tweaks to come that will improve/change outcomes.
</div>

Introduction
---
**RKnot** is a simulation architecture for viral spread that aims to:

1. Faithfully replicate spread in real world conditions
2. Investigate the impacts of policy decisions and other interventions
3. Provide visualization tools for ease-of-presentation

**RKnot** attempts to distinguish itself from prevailing models by:

+ allowing for customized population sizes and demographics
+ supporting more realistic movement and contact patterns
+ modulating transmission risk to account for varying subject behavior and location properties
+ influencing interactions via an array of interventions that can be used in any combination

**RKnot** utilizes parallelization via [Ray](https://docs.ray.io/en/master/index.html) and JIT-compilation via [Numba](https://numba.pydata.org/) for performance improvements amd [Matplotlib](https://matplotlib.org/) for visualizaitons.

<div class="alert alert-info">
    Note
    
    Documentation examples focus on **sars-cov-2**, however, **RKnot** can be used to simulate <em>any</em> virus, or anything that spreads like a virus.
</div>


### Basic Example ###

A simulation and visualization can be completed in a few quick lines of code. The user need only specify a dictionary describing the population, `group`, and a handful of parameters describing the simulation space and viral characteristics.

Below we simulate:

* a population of 1,000 subjects,
* beginning with five initially infected;
<p></p>
* a density of 1 subject per location
* a maximum simulation length of 150 days 
    * *the simulation will automatically stop when there are no more infections*
<p></p>
+ an initial reproduction number, $R_0$, of 3
+ an infection duration of 14 days
+ an immunity duration of 365

```python
from rknot import Sim, Chart

group = {'n': 1000, 'n_inf': 5}
params = {'R0': 3,'imndur': 365, 'infdur': 14, 'density': 1, 'days': 150}


sim = Sim(groups=group, **params)
sim.run()

chart = Chart(sim)
chart.to_html5_video()
```

In [4]:
from rknot import Sim, Chart

group = {'n': 1000, 'n_inf': 5}
params = {'R0': 3,'imndur': 365, 'infdur': 14, 'density': 1, 'days': 150}


sim = Sim(groups=group, **params)
sim.run(kill=False)

chart = Chart(sim, dotsize=2, reso_format='web').save('vids/intro/basic_example.mp4')

/Users/spindicate/Documents/programming/rknot/rknot/animate/main.py:226: UserWarning: When `reso_format='web', you must call the save method at the same time as instantiating the object, or it will hang.
  warnings.warn(msg)


/Users/spindicate/Documents/programming/rknot/rknot/animate/subplots.py:504: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  self.ax.set_xlim(0, xlim)
/Users/spindicate/Documents/programming/rknot/rknot/animate/subplots.py:510: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  self.depper.axes.set_xlim(0, xlim)


/Users/spindicate/Documents/programming/rknot/rknot/animate/subplots.py:504: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  self.ax.set_xlim(0, xlim)
/Users/spindicate/Documents/programming/rknot/rknot/animate/subplots.py:510: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  self.depper.axes.set_xlim(0, xlim)


In [5]:
from datetime import datetime as dt
from rknot.helpers import picksave, pickload
file = 'vids/intro/basic_example'

if SAVE:
    delattr(sim, 'server')
    delattr(sim, 'workers')
    sims = pickload(file)
    sims[dt.now()] = sim
    picksave(file, sims)

    sims = pickload(file)
    run_date, sim = sorted(sims.items())[-1] # Get the most recent saved sim
else:
    sims = pickload(file)
    run_date, sim = sorted(sims.items())[-1] # Get the most recent saved sim
    res = get_results(sim)

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/intro/basic_example.mp4' controls>alternative text</video>
<p></p>

In [10]:
from rknot.helpers import get_results
from rknot.notebook import md, results_table

md(results_table(get_results(sim)))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   52.1% | 
| HIT      | 72.7%      | 
| Total      | 92.2%      | 
| Fatalities | 0.00%      | 
| % > 70 | nan%      | 
| IFR | 0.00%        | 
| Days to Peak |   36 | 


As per the chart, this simulation results in a peak at 29 days, with 53% of the population infected at the peak and a Herd Immunity Threshold of 70%. In total 93% of the population was infected and 0.4% of the population, or 4 subjects, died.

### Next Steps ###

+ [Here you can explore how viral spread theory is incorporate into *RKnot*.](theory.ipynb)

+ [Here you can learn about the core concepts on which the simulation architecture is built.](concepts.ipynb)

Or you can jump right into the example simulations we have built and explore how different properities impact spread:

+ [Factors Influencing Spread](sir.ipynb)
+ [Impact of Dynamce Transmission Risk](load.ipynb)


In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
from rknot.notebook import move_to_doc_folder
move_to_doc_folder('intro.ipynb')